In [2]:
print('hello')

hello


In [9]:
import openai
from dotenv import load_dotenv
import os



In [10]:
load_dotenv()

# Now you can access your environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
import requests

# Function to fetch a random cat fact
def get_cat_fact(_: str) -> str:
    try:
        response = requests.get("https://catfact.ninja/fact")
        if response.status_code == 200:
            data = response.json()
            return data.get("fact", "Couldn't find a cat fact.")
        return f"Failed to get cat fact. Status code: {response.status_code}"
    except Exception as e:
        return f"API call failed: {e}"

# Wrap it as a LangChain Tool
cat_fact_tool = Tool(
    name="CatFactFetcher",
    func=get_cat_fact,
    description="Provides a random cat fact when someone asks for one."
)

# Set up the agent
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    tools=[cat_fact_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example usage
print(agent.run("Hi dont tell me a cat fact"))


: 